In [ ]:
import pandas as pd
import re
import sentence_transformers
import numpy as np

In [ ]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=be294f695bb9bc16681f8553e87cf0f0c95251ee1915860a1ef783ed1fbe68ee
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
df1=pd.read_csv('stockx_counterfeit_data.csv')
df2=pd.read_csv('shoe_meta5.csv')
dataset=pd.concat([df1,df2])
dataset = dataset.sample(frac = 1)
dataset.head()

,index,Title,Price,Brand,Unnamed: 0,Company,Rating
801,801.0,Nike Dunk High Tan Suede White,CA$147,Nike,NaN,NaN,NaN
326,326.0,Nike Air Max 270 Black White (W),CA$129,Nike,NaN,NaN,NaN
4836,NaN,"Women's Racer Tr21 Running Shoe, Black/Grey/Pr...",$99.97,NaN,4836.0,adidas,No reviews
1785,NaN,Mens Lite Plus 3.0 Running Shoe,$71.66,NaN,1785.0,Reebok,4.0
4765,NaN,"Men's Fresh Foam 510 V6 Trail Running Shoe, Bl...",$94.97,NaN,4765.0,New Balance,No reviews


In [ ]:
dataset=dataset.drop(columns='index')
dataset.head()

,Title,Price,Brand,Unnamed: 0,Company,Rating
801,Nike Dunk High Tan Suede White,CA$147,Nike,NaN,NaN,NaN
326,Nike Air Max 270 Black White (W),CA$129,Nike,NaN,NaN,NaN
4836,"Women's Racer Tr21 Running Shoe, Black/Grey/Pr...",$99.97,NaN,4836.0,adidas,No reviews
1785,Mens Lite Plus 3.0 Running Shoe,$71.66,NaN,1785.0,Reebok,4.0
4765,"Men's Fresh Foam 510 V6 Trail Running Shoe, Bl...",$94.97,NaN,4765.0,New Balance,No reviews


In [ ]:
for i in range(len(dataset)):
    if dataset['Rating'].values[i]=='New to Amazon':
        dataset['Rating'].values[i]=3.7
    if dataset['Rating'].values[i]=='No reviews':
        dataset['Rating'].values[i]=4.2

In [ ]:
Prices=[]
cleaned_pricem=[]
for k in dataset['Price']:
    #k.replace('$','')
    #print(k)
    r1=re.sub(r'[A-Za-z]','', k)
    r2=re.sub(r'[$]','',r1)
    r3=re.sub(r'[,]','',r2)
    r4=re.sub(r'[--]','',r3)
    try:
        cleaned_pricem.append(int(r4))
    except:
         cleaned_pricem.append(180)

In [ ]:
dataset['cleaned_price']=cleaned_pricem

In [ ]:
Titles_cleaned=[]
for test_str in dataset['Title']:
    res = re.sub(r'[^\w\s]', '', test_str)
    Titles_cleaned.append(res)
dataset['Titles_cleaned']= Titles_cleaned
dataset.head()

,Title,Price,Brand,Unnamed: 0,Company,Rating,cleaned_price,Titles_cleaned
801,Nike Dunk High Tan Suede White,CA$147,Nike,NaN,NaN,NaN,147,Nike Dunk High Tan Suede White
326,Nike Air Max 270 Black White (W),CA$129,Nike,NaN,NaN,NaN,129,Nike Air Max 270 Black White W
4836,"Women's Racer Tr21 Running Shoe, Black/Grey/Pr...",$99.97,NaN,4836.0,adidas,4.2,180,Womens Racer Tr21 Running Shoe BlackGreyPrelov...
1785,Mens Lite Plus 3.0 Running Shoe,$71.66,NaN,1785.0,Reebok,4.0,180,Mens Lite Plus 30 Running Shoe
4765,"Men's Fresh Foam 510 V6 Trail Running Shoe, Bl...",$94.97,NaN,4765.0,New Balance,4.2,180,Mens Fresh Foam 510 V6 Trail Running Shoe Blac...


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#Our sentences we like to encode
sentences = Titles_cleaned
sent_list=[]

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)
embed_list=[]

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")
    sent_list.append(sentence)
    embed_list.append(embedding)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
dataset['title_vector'] = embed_list
dataset

,Title,Price,Brand,Unnamed: 0,Company,Rating,cleaned_price,Titles_cleaned,title_vector
801,Nike Dunk High Tan Suede White,CA$147,Nike,NaN,NaN,NaN,147,Nike Dunk High Tan Suede White,"[-0.02241408, 0.10138695, 0.01466169, -0.02344..."
326,Nike Air Max 270 Black White (W),CA$129,Nike,NaN,NaN,NaN,129,Nike Air Max 270 Black White W,"[-0.05129999, 0.033587307, -0.07079396, 0.0481..."
4836,"Women's Racer Tr21 Running Shoe, Black/Grey/Pr...",$99.97,NaN,4836.0,adidas,4.2,180,Womens Racer Tr21 Running Shoe BlackGreyPrelov...,"[-0.11990807, 0.009859877, -0.035551276, 0.027..."
1785,Mens Lite Plus 3.0 Running Shoe,$71.66,NaN,1785.0,Reebok,4.0,180,Mens Lite Plus 30 Running Shoe,"[-0.05286994, 0.014577098, -0.051006313, 0.033..."
4765,"Men's Fresh Foam 510 V6 Trail Running Shoe, Bl...",$94.97,NaN,4765.0,New Balance,4.2,180,Mens Fresh Foam 510 V6 Trail Running Shoe Blac...,"[-0.039663877, -0.009417292, -0.07608739, 0.04..."
...,...,...,...,...,...,...,...,...,...
23,Nike Dunk Low LX Pink Foam (Women's),CA$201,Nike,NaN,NaN,NaN,201,Nike Dunk Low LX Pink Foam Womens,"[-0.0016946639, -0.030515285, 0.027935049, -0...."
5001,Training Footwear Men's Speed 22 Tr Shoes Citr...,$140.00,NaN,5001.0,Reebok,4.2,180,Training Footwear Mens Speed 22 Tr Shoes Citru...,"[-0.047813147, 0.035059184, -0.04351007, 0.050..."
3012,Men's 466302,$798.64,NaN,3012.0,adidas,5.0,180,Mens 466302,"[-0.17201081, -0.05661849, -0.057677817, 0.038..."
4430,"Men's Fresh Foam Arishi V4 Running Shoe, Grey/...",$86.95,NaN,4430.0,New Balance,5.0,180,Mens Fresh Foam Arishi V4 Running Shoe GreyGum...,"[-0.037938308, 0.0514632, 0.0076576886, 0.0150..."


In [ ]:
dataset['title_vector'].shape

(8423,)

In [ ]:
df = pd.read_csv("embedde_sandra.csv")

FileNotFoundError: ignored

In [ ]:
dataset['label'] = df['Label']

In [ ]:
dataset['Brand'] = df['Brand']

In [ ]:
dataset['Rating'] = df['Rating']

In [ ]:
with open('LEE.pkl', 'rb') as f:
    le = pickle.load(f)

In [ ]:
from sklearn import preprocessing
import pandas as pd

In [ ]:
df = pd.read_csv("meta_data.csv")

In [ ]:
dataset

,Title,Price,Brand,Unnamed: 0,Company,Rating,cleaned_price,Titles_cleaned,title_vector
801,Nike Dunk High Tan Suede White,CA$147,Nike,NaN,NaN,NaN,147,Nike Dunk High Tan Suede White,"[-0.02241408, 0.10138695, 0.01466169, -0.02344..."
326,Nike Air Max 270 Black White (W),CA$129,Nike,NaN,NaN,NaN,129,Nike Air Max 270 Black White W,"[-0.05129999, 0.033587307, -0.07079396, 0.0481..."
4836,"Women's Racer Tr21 Running Shoe, Black/Grey/Pr...",$99.97,NaN,4836.0,adidas,4.2,180,Womens Racer Tr21 Running Shoe BlackGreyPrelov...,"[-0.11990807, 0.009859877, -0.035551276, 0.027..."
1785,Mens Lite Plus 3.0 Running Shoe,$71.66,NaN,1785.0,Reebok,4.0,180,Mens Lite Plus 30 Running Shoe,"[-0.05286994, 0.014577098, -0.051006313, 0.033..."
4765,"Men's Fresh Foam 510 V6 Trail Running Shoe, Bl...",$94.97,NaN,4765.0,New Balance,4.2,180,Mens Fresh Foam 510 V6 Trail Running Shoe Blac...,"[-0.039663877, -0.009417292, -0.07608739, 0.04..."
...,...,...,...,...,...,...,...,...,...
23,Nike Dunk Low LX Pink Foam (Women's),CA$201,Nike,NaN,NaN,NaN,201,Nike Dunk Low LX Pink Foam Womens,"[-0.0016946639, -0.030515285, 0.027935049, -0...."
5001,Training Footwear Men's Speed 22 Tr Shoes Citr...,$140.00,NaN,5001.0,Reebok,4.2,180,Training Footwear Mens Speed 22 Tr Shoes Citru...,"[-0.047813147, 0.035059184, -0.04351007, 0.050..."
3012,Men's 466302,$798.64,NaN,3012.0,adidas,5.0,180,Mens 466302,"[-0.17201081, -0.05661849, -0.057677817, 0.038..."
4430,"Men's Fresh Foam Arishi V4 Running Shoe, Grey/...",$86.95,NaN,4430.0,New Balance,5.0,180,Mens Fresh Foam Arishi V4 Running Shoe GreyGum...,"[-0.037938308, 0.0514632, 0.0076576886, 0.0150..."


AttributeError: ignored

In [ ]:
type(k)

method

In [ ]:

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
B=le.fit(df['Brand'])
df['Brand_enc']=le.transform(df['Brand'])

In [ ]:
pickle.dump(le, open('lefinal', 'wb'))

In [ ]:
dataset

,Title,Price,Brand,Unnamed: 0,Company,Rating,cleaned_price,Titles_cleaned,title_vector
801,Nike Dunk High Tan Suede White,CA$147,Nike,NaN,NaN,NaN,147,Nike Dunk High Tan Suede White,"[-0.02241408, 0.10138695, 0.01466169, -0.02344..."
326,Nike Air Max 270 Black White (W),CA$129,Nike,NaN,NaN,NaN,129,Nike Air Max 270 Black White W,"[-0.05129999, 0.033587307, -0.07079396, 0.0481..."
4836,"Women's Racer Tr21 Running Shoe, Black/Grey/Pr...",$99.97,NaN,4836.0,adidas,4.2,180,Womens Racer Tr21 Running Shoe BlackGreyPrelov...,"[-0.11990807, 0.009859877, -0.035551276, 0.027..."
1785,Mens Lite Plus 3.0 Running Shoe,$71.66,NaN,1785.0,Reebok,4.0,180,Mens Lite Plus 30 Running Shoe,"[-0.05286994, 0.014577098, -0.051006313, 0.033..."
4765,"Men's Fresh Foam 510 V6 Trail Running Shoe, Bl...",$94.97,NaN,4765.0,New Balance,4.2,180,Mens Fresh Foam 510 V6 Trail Running Shoe Blac...,"[-0.039663877, -0.009417292, -0.07608739, 0.04..."
...,...,...,...,...,...,...,...,...,...
23,Nike Dunk Low LX Pink Foam (Women's),CA$201,Nike,NaN,NaN,NaN,201,Nike Dunk Low LX Pink Foam Womens,"[-0.0016946639, -0.030515285, 0.027935049, -0...."
5001,Training Footwear Men's Speed 22 Tr Shoes Citr...,$140.00,NaN,5001.0,Reebok,4.2,180,Training Footwear Mens Speed 22 Tr Shoes Citru...,"[-0.047813147, 0.035059184, -0.04351007, 0.050..."
3012,Men's 466302,$798.64,NaN,3012.0,adidas,5.0,180,Mens 466302,"[-0.17201081, -0.05661849, -0.057677817, 0.038..."
4430,"Men's Fresh Foam Arishi V4 Running Shoe, Grey/...",$86.95,NaN,4430.0,New Balance,5.0,180,Mens Fresh Foam Arishi V4 Running Shoe GreyGum...,"[-0.037938308, 0.0514632, 0.0076576886, 0.0150..."


In [ ]:
import pickle
with open('LEfin.pkl','wb') as f:
    pickle.dump(B,f)

In [ ]:
dataset['label'] = df['Label']

In [ ]:
dataset['Brand'] = df['Brand_enc']

In [ ]:
dataset['Rating'] = df['Rating']

In [ ]:
dataset

,Title,Price,Brand,Unnamed: 0,Company,Rating,cleaned_price,Titles_cleaned,title_vector,label
801,Nike Dunk High Tan Suede White,CA$147,6,NaN,NaN,4.2,147,Nike Dunk High Tan Suede White,"[-0.02241408, 0.10138695, 0.01466169, -0.02344...",1
326,Nike Air Max 270 Black White (W),CA$129,5,NaN,NaN,4.0,129,Nike Air Max 270 Black White W,"[-0.05129999, 0.033587307, -0.07079396, 0.0481...",1
4836,"Women's Racer Tr21 Running Shoe, Black/Grey/Pr...",$99.97,6,4836.0,adidas,4.2,180,Womens Racer Tr21 Running Shoe BlackGreyPrelov...,"[-0.11990807, 0.009859877, -0.035551276, 0.027...",1
1785,Mens Lite Plus 3.0 Running Shoe,$71.66,0,1785.0,Reebok,4.2,180,Mens Lite Plus 30 Running Shoe,"[-0.05286994, 0.014577098, -0.051006313, 0.033...",1
4765,"Men's Fresh Foam 510 V6 Trail Running Shoe, Bl...",$94.97,4,4765.0,New Balance,4.2,180,Mens Fresh Foam 510 V6 Trail Running Shoe Blac...,"[-0.039663877, -0.009417292, -0.07608739, 0.04...",1
...,...,...,...,...,...,...,...,...,...,...
23,Nike Dunk Low LX Pink Foam (Women's),CA$201,6,NaN,NaN,2.0,201,Nike Dunk Low LX Pink Foam Womens,"[-0.0016946639, -0.030515285, 0.027935049, -0....",0
5001,Training Footwear Men's Speed 22 Tr Shoes Citr...,$140.00,6,5001.0,Reebok,4.2,180,Training Footwear Mens Speed 22 Tr Shoes Citru...,"[-0.047813147, 0.035059184, -0.04351007, 0.050...",1
3012,Men's 466302,$798.64,2,3012.0,adidas,4.2,180,Mens 466302,"[-0.17201081, -0.05661849, -0.057677817, 0.038...",1
4430,"Men's Fresh Foam Arishi V4 Running Shoe, Grey/...",$86.95,2,4430.0,New Balance,4.2,180,Mens Fresh Foam Arishi V4 Running Shoe GreyGum...,"[-0.037938308, 0.0514632, 0.0076576886, 0.0150...",1


In [ ]:
d = list(dataset['title_vector'])

In [ ]:
list_vec_arr=np.array(d)

In [ ]:
list_vec_arr

array([[-0.02241408,  0.10138695,  0.01466169, ..., -0.11260628,
        -0.03633433,  0.02304825],
       [-0.05129999,  0.03358731, -0.07079396, ..., -0.08654219,
        -0.09020909,  0.00400454],
       [-0.11990807,  0.00985988, -0.03555128, ..., -0.15588738,
        -0.01774113, -0.01419554],
       ...,
       [-0.17201081, -0.05661849, -0.05767782, ..., -0.01086937,
        -0.04836473, -0.04440611],
       [-0.03793831,  0.0514632 ,  0.00765769, ..., -0.11424184,
         0.00570588,  0.0515113 ],
       [-0.10790423,  0.04099118,  0.01401556, ..., -0.06198583,
        -0.01038798, -0.04914229]], dtype=float32)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=12)
X_p = pca.fit(list_vec_arr)
X_pca = pca.transform(list_vec_arr)

In [ ]:
pickle.dump(pca, open('pcafinal', 'wb'))

In [ ]:
dataset['title_vector'] = X_pca

In [ ]:
dataset

,Title,Price,Brand,Unnamed: 0,Company,Rating,cleaned_price,Titles_cleaned,title_vector,label
801,Nike Dunk High Tan Suede White,CA$147,6,NaN,NaN,4.2,147,Nike Dunk High Tan Suede White,0.138064,1
326,Nike Air Max 270 Black White (W),CA$129,5,NaN,NaN,4.0,129,Nike Air Max 270 Black White W,0.167407,1
4836,"Women's Racer Tr21 Running Shoe, Black/Grey/Pr...",$99.97,6,4836.0,adidas,4.2,180,Womens Racer Tr21 Running Shoe BlackGreyPrelov...,-0.095016,1
1785,Mens Lite Plus 3.0 Running Shoe,$71.66,0,1785.0,Reebok,4.2,180,Mens Lite Plus 30 Running Shoe,-0.239989,1
4765,"Men's Fresh Foam 510 V6 Trail Running Shoe, Bl...",$94.97,4,4765.0,New Balance,4.2,180,Mens Fresh Foam 510 V6 Trail Running Shoe Blac...,-0.102694,1
...,...,...,...,...,...,...,...,...,...,...
23,Nike Dunk Low LX Pink Foam (Women's),CA$201,6,NaN,NaN,2.0,201,Nike Dunk Low LX Pink Foam Womens,-0.016850,0
5001,Training Footwear Men's Speed 22 Tr Shoes Citr...,$140.00,6,5001.0,Reebok,4.2,180,Training Footwear Mens Speed 22 Tr Shoes Citru...,-0.177260,1
3012,Men's 466302,$798.64,2,3012.0,adidas,4.2,180,Mens 466302,0.173103,1
4430,"Men's Fresh Foam Arishi V4 Running Shoe, Grey/...",$86.95,2,4430.0,New Balance,4.2,180,Mens Fresh Foam Arishi V4 Running Shoe GreyGum...,-0.111917,1


In [ ]:
df1['title_pca']=df1['title_vector']

In [ ]:
df1=df1.drop(columns='title_vector')

In [ ]:
df1.to_csv('t2_model_data.csv')

In [ ]:
dataset = pd.read_csv('t2_model_data.csv')

In [ ]:
X=dataset[['cleaned_price','Rating','title_pca','Brand']]
y = dataset['label']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size = 0.1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense


In [ ]:
classifier = Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 4))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN | means applying SGD on the whole ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100,verbose = 0)

score, acc = classifier.evaluate(X_val, y_val,
                            batch_size=10)
print('Train score:', score)
print('Train accuracy:', acc)

85/85 [==============================] - 0s 2ms/step - loss: 0.1393 - accuracy: 0.9300
Train score: 0.13931478559970856
Train accuracy: 0.9300118684768677


In [ ]:
y_pred = classifier.predict(X_test)
y_test = (y_test > 0.5)

print('*'*20)
score, acc = classifier.evaluate(X_test, y_test,
                            batch_size=10)
print('Test score:', score)
print('Test accuracy:', acc)

22/22 [==============================] - 0s 1ms/step
********************
69/69 [==============================] - 0s 2ms/step - loss: 0.1590 - accuracy: 0.9122
Test score: 0.15896765887737274
Test accuracy: 0.9121522903442383


In [ ]:
classifier.save("meta_model_fine",save_format="h5")

In [ ]:
classifier.predict('')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model("meta_model_fine")

In [ ]:
y_pred = model.predict(X_val)

27/27 [==============================] - 0s 1ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
preds = [round(x[0]) for x in classifier.predict(X_test)]

22/22 [==============================] - 0s 2ms/step


In [ ]:
cm = confusion_matrix(y_test,preds)
cm_df = pd.DataFrame(cm, columns = ['pred_rep','pred_auth'],
                     index=['actual_rep','actual_auth'])
cm_df

,pred_rep,pred_auth
actual_rep,170,44
actual_auth,16,453


In [ ]:
TP, TN, FP, FN = cm[0,0], cm[1,1], cm[1,0], cm[0,1]
precision = TP/(TP+FP)
print(f'Precision: {precision}')

Precision: 0.9139784946236559


In [ ]:
cl_report= classification_report(y_test,preds)

In [ ]:
 print("\nCR by library method=\n",cl_report)


CR by library method=
               precision    recall  f1-score   support

       False       0.91      0.79      0.85       214
        True       0.91      0.97      0.94       469

    accuracy                           0.91       683
   macro avg       0.91      0.88      0.89       683
weighted avg       0.91      0.91      0.91       683

